<a href="https://colab.research.google.com/github/cristinarainich/DDCS/blob/main/comorbidity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import pylab as pl
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy.stats as stats

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
working_path = Path('/content/gdrive/MyDrive/KKNV/comorbidity')

In [ ]:
!ls /content/gdrive/MyDrive/KKNV/comorbidity

data100.csv   data135.csv  data175.csv	data23.csv  data48.csv	data78.csv
data101.csv   data136.csv  data17.csv	data24.csv  data49.csv	data79.csv
data102.csv   data139.csv  data183.csv	data25.csv  data51.csv	data7.csv
data103.csv   data13.csv   data184.csv	data27.csv  data52.csv	data80.csv
data104.csv   data140.csv  data187.csv	data28.csv  data54.csv	data81.csv
data106.csv   data141.csv  data189.csv	data_2.csv  data55.csv	data82.csv
data109a.csv  data142.csv  data18.csv	data33.csv  data56.csv	data83.csv
data109.csv   data143.csv  data192.csv	data34.csv  data57.csv	data86.csv
data110.csv   data145.csv  data193.csv	data36.csv  data58.csv	data87.csv
data111.csv   data149.csv  data196.csv	data38.csv  data59.csv	data88.csv
data112.csv   data153.csv  data_1.csv	data39.csv  data60.csv	data89.csv
data113.csv   data155.csv  data202.csv	data3.csv   data62.csv	data92.csv
data116.csv   data157.csv  data204.csv	data40.csv  data65.csv	data93.csv
data118.csv   data158.csv  data205.csv	data41.csv  data

In [ ]:
## function to calculate the results on prosopagnosia test
def PI_test(data):
  pi_subset = data[['PI20_01', 'PI20_02', 'PI20_03', 'PI20_04', 'PI20_05',
       'PI20_06', 'PI20_07', 'PI20_08', 'PI20_09', 'PI20_10', 'PI20_11',
       'PI20_12', 'PI20_13', 'PI20_14', 'PI20_15', 'PI20_16', 'PI20_17',
       'PI20_18', 'PI20_19', 'PI20_20']]
  tasks = data['Task_Name'].values.tolist()
  num = tasks.index('PI20')

  values = pi_subset.values[num].tolist()
  values = [int(i) for i in values]
  positions = [7, 8, 12, 16, 18]
  for i in positions:
    new_res = 6 - values[i]
    values[i] = new_res

  final_score = 0
  for each in values:
    final_score = final_score + each
  if 65 <= final_score <= 74:
    diagnosis = 'Mild prosopagnosia'
  elif 74 < final_score <= 84:
    diagnosis = "Moderate prosopagnosia"
  elif final_score > 84:
    diagnosis = 'Severe prosopagnosia'
  else:
    diagnosis = 'No prosopagnosia'

  return final_score, diagnosis


## function to calculate the results on autism test
def AQ_test(data):
  aq_subset = data[['AQ_Q01', 'AQ_Q02', 'AQ_Q03', 'AQ_Q04', 'AQ_Q05',
        'AQ_Q06', 'AQ_Q07', 'AQ_Q08', 'AQ_Q09', 'AQ_Q10', 'AQ_Q11', 'AQ_Q12',
        'AQ_Q13', 'AQ_Q14', 'AQ_Q15', 'AQ_Q16', 'AQ_Q17', 'AQ_Q18', 'AQ_Q19',
        'AQ_Q20', 'AQ_Q21', 'AQ_Q22', 'AQ_Q23', 'AQ_Q24', 'AQ_Q25', 'AQ_Q26',
        'AQ_Q27', 'AQ_Q28', 'AQ_Q29', 'AQ_Q30', 'AQ_Q31', 'AQ_Q32', 'AQ_Q33',
        'AQ_Q34', 'AQ_Q35', 'AQ_Q36', 'AQ_Q37', 'AQ_Q38', 'AQ_Q39', 'AQ_Q40',
        'AQ_Q41', 'AQ_Q42', 'AQ_Q43', 'AQ_Q44', 'AQ_Q45', 'AQ_Q46', 'AQ_Q47',
        'AQ_Q48', 'AQ_Q49', 'AQ_Q50']]
  tasks = data['Task_Name'].values.tolist()
  num = tasks.index('AQ')

  values = aq_subset.values[num].tolist()
  values = [int(i) for i in values]

  positions1 = [1, 3, 8, 10, 11, 14, 15, 17, 24, 25, 27, 28, 29, 30, 31, 32, 34, 36, 37, 38, 40, 44, 47, 48, 49, 50]
  positions2 = [2, 4, 5, 6, 7, 9, 12, 13, 16, 18, 19, 20, 21, 22, 23, 26, 33, 35, 39, 41, 42, 43, 45, 46]
  for j in range(0, len(positions1)):
    positions1[j] = positions1[j] - 1
  for j in range(0, len(positions2)):
    positions2[j] = positions2[j] - 1

  for i in range(0, 49):
    if i in positions1:
      if values[i] == 3 or values[i] == 4:
        res = 1
      elif values[i] == 1 or values[i] == 2:
        res = 0
    elif i in positions2:
      if values[i] == 3 or values[i] == 4:
        res = 0
      elif values[i] == 1 or values[i] == 2:
        res = 1
    values[i] = res

  final_score = 0
  for each in values:
    final_score = final_score + each
  if final_score >= 32:
    diagnosis = 'Autistic traits'
  else:
    diagnosis = 'No autistic traits'

  return final_score, diagnosis

## function to calculate the results on ADHD test
def ASR_test(data):
  asr_subset = data[['ASRS_Q1', 'ASRS_Q2', 'ASRS_Q3', 'ASRS_Q4',
                     'ASRS_Q5', 'ASRS_Q6']]
  tasks = data['Task_Name'].values.tolist()
  num = tasks.index('ARSR-v1.1.')

  values = asr_subset.values[num].tolist()
  values = [int(i) for i in values]

  for i in range(0, len(values)):
    if i in [0, 1, 2]:
      if values[i] == 3 or values[i] == 4 or values[i] == 5:
        res = 1
      else:
        res = 0
    elif i in [3, 4, 5]:
      if values[i] == 4 or values[i] == 5:
        res = 1
      else:
        res = 0
    values[i] = res

  final_score = 0
  for each in values:
    final_score = final_score + each
  if final_score >= 4:
    diagnosis = 'ADHD symptoms'
  else:
    diagnosis = 'No ADHD traits'

  return final_score, diagnosis


## function to calculate the results on ADHD test
def EHI_test(data):
  ehi_subset = data[['handedness_01', 'handedness_02', 'handedness_03',
       'handedness_04']]
  tasks = data['Task_Name'].values.tolist()
  num = tasks.index('Edinburgh Handedness Inventory')

  values = ehi_subset.values[num].tolist()
  values = [int(i) for i in values]

  for i in range(0, len(values)):
    if values[i] == 5:
      res = 100
    elif values[i] == 4:
      res = 50
    elif values[i] == 3:
      res = 0
    elif values[i] == 2:
      res = -50
    elif values[i] == 1:
      res = -100
    values[i] = res

  final_score = 0
  for each in values:
    final_score = final_score + each
  final_score = final_score / 4
  if -100 <= final_score <= -61:
    diagnosis = 'left-handed'
  elif -60 <= final_score <= 60:
    diagnosis = 'mixed'
  elif 61 <= final_score <= 100:
    diagnosis = 'right-handed'

  return final_score, diagnosis

In [ ]:
def run_all_tests(data_number):
  file_name = f'data{data_number}.csv'
  data_path = working_path / file_name

  if not data_path.exists():
        print(f"File {file_name} not found.")
        return

  data = pd.read_csv(data_path, index_col=None)


  print(f"\nRunning tests on {file_name}:\n")
  print("PI:", PI_test(data))
  print("AQ:", AQ_test(data))
  print("ASR:", ASR_test(data))
  print("EHI:", EHI_test(data))

In [ ]:
data_points = [216, 221, 205, 222, 223] # insert the labvanced IDs in this line
for number in data_points:
  run_all_tests(number)


Running tests on data216.csv:

PI: (54, 'No prosopagnosia')
AQ: (28, 'No autistic traits')
ASR: (4, 'ADHD symptoms')
EHI: (87.5, 'right-handed')

Running tests on data221.csv:

PI: (28, 'No prosopagnosia')
AQ: (20, 'No autistic traits')
ASR: (0, 'No ADHD traits')
EHI: (100.0, 'right-handed')

Running tests on data205.csv:

PI: (30, 'No prosopagnosia')
AQ: (17, 'No autistic traits')
ASR: (0, 'No ADHD traits')
EHI: (87.5, 'right-handed')

Running tests on data222.csv:

PI: (41, 'No prosopagnosia')
AQ: (27, 'No autistic traits')
ASR: (2, 'No ADHD traits')
EHI: (62.5, 'right-handed')

Running tests on data223.csv:

PI: (62, 'No prosopagnosia')
AQ: (12, 'No autistic traits')
ASR: (2, 'No ADHD traits')
EHI: (62.5, 'right-handed')
